<div class=title>
<strong><center><font size="3">DAY 23 - IYKRA</font></center>
<hr>
<center><font size="5"><strong>Data Science Bootcamp</strong></font></center>
<center><font size="4"><strong>Applied Machine Learning: Server</strong></font></center>
<hr>
<p style="text-align:center">Author</p>
<center>Joshua Effendi and Gilang Mahardika</center>
<p style="text-align:center">Date:</p>
<center>20 November 2019</center></strong>
</div>

In [1]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import HTML

def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

<div class=title1>
    <b><font size='3'>Import Libraries</font></b>
</div>
<div class=title2>
    <b><font size='2'>Import libraries that are needed to run the server</font></b>
</div>

In [ ]:
from flask import Flask, request, jsonify #To handle the communication
import pickle #To open imported model in the server
app = Flask(__name__)

<div class=title1>
    <b><font size='3'>Load Model</font></b>
</div>
<div class=title2>
    <b><font size='2'>Load existing model that is saved in a pickle file</font></b>
</div>

In [ ]:
model = pickle.load(open('model_xgb.pkl','rb'))

<div class=title1>
    <b><font size='3'>Setup Connection Method System</font></b>
</div>
<div class=title2>
    <b><font size='2'>Setup the connection of the server using the method POST to send data from the server</font></b>
</div>

In [ ]:
@app.route('/api',methods=['POST'])

<div class=title1>
    <b><font size='3'>Function of Model Prediction</font></b>
</div>
<div class=title2>
    <b><font size='2'>Create a function that give prediction from a loaded model that are requested from the clients' side</font></b>
</div>

In [ ]:
def predict():
    # Get the data from the POST request.
    data = request.get_json(force=True)
    
    # Transform JSON into DataFrame
    data = pd.DataFrame.from_dict(data)
    
    def age_group(age):
        if age < 35:
            return 'Productive'
        if age < 50:
            return 'Mature'
        else:
            return 'Old'
    
    # Apply function of age_group to DataFrame
    data['age_group'] = data['age'].apply(age_group)
    
    # Prepare data by teir data types
    dummy = data.select_dtypes(exclude = ['int', 'int64', 'float64'])
    integer = data.select_dtypes(include = ['int', 'int64', 'float64'])
    
    # Do OHE on dummy
    dummy2 = pd.get_dummies(dummy, drop_first = True)
    
    # Join the data from the integer part and also the OHE part
    data = pd.concat([integer, dummy2], axis = 1)
    
    # Setup StandardScaler
    SC = StandardScaler()
    
    #Do StandardScaler to Data
    data = SC.fit_transform(data)

    # Make prediction using model loaded from disk as per the data.
    prediction = model.predict(data)

    # Take the first value of prediction
    output = prediction.tolist()

    return jsonify(output)

<div class=title1>
    <b><font size='3'>Setup the Port Connection</font></b>
</div>
<div class=title2>
    <b><font size='2'>Setup the desired port and try to run the server, if it's disconnect, print "Server got disconnected"</font></b>
</div>

In [ ]:
if __name__ == '__main__':
    try:
        app.run(port=1234, debug=True)
    except:
        print("Server is exited unexpectedly. Please contact server admin.")